In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import tifffile
from careamics_portfolio import PortfolioManager
from pytorch_lightning import Trainer

from careamics import CAREamicsModule
from careamics.lightning_datamodule import (
    CAREamicsPredictDataModule,
    CAREamicsTrainDataModule,
)
from careamics.lightning_prediction import CAREamicsPredictionLoop
from careamics.utils.metrics import psnr

### Import Dataset Portfolio

In [ ]:
# Explore portfolio
portfolio = PortfolioManager()
print(portfolio.denoising)

In [ ]:
# Download and unzip the files
root_path = Path("data")
files = portfolio.denoising.N2V_BSD68.download(root_path)
print(f"List of downloaded files: {files}")

In [ ]:
data_path = Path(root_path / "denoising-N2V_BSD68.unzip/BSD68_reproducibility_data")
train_path = data_path / "train"
val_path = data_path / "val"
test_path = data_path / "test" / "images"
gt_path = data_path / "test" / "gt"

train_path.mkdir(parents=True, exist_ok=True)
val_path.mkdir(parents=True, exist_ok=True)
test_path.mkdir(parents=True, exist_ok=True)
gt_path.mkdir(parents=True, exist_ok=True)

### Visualize training data

In [ ]:
train_image = tifffile.imread(next(iter(train_path.rglob("*.tiff"))))[0]
print(f"Train image shape: {train_image.shape}")
plt.imshow(train_image, cmap="gray")

### Visualize validation data

In [ ]:
val_image = tifffile.imread(next(iter(val_path.rglob("*.tiff"))))[0]
print(f"Validation image shape: {val_image.shape}")
plt.imshow(val_image, cmap="gray")

### Initialize the Model

Create a Pytorch Lightning module

Please take as look at the [documentation](https://careamics.github.io) to see the full list of parameters and configuration options

In [ ]:
# N2V2 requires changes to the UNet model and to the Dataset (augmentations)
use_n2v2 = False  # change to True to use N2V2

In [ ]:
model = CAREamicsModule(
    algorithm="n2v",
    loss="n2v",
    architecture="UNet",
    model_parameters={"n2v2": use_n2v2},
    optimizer_parameters={"lr": 1e-3},
    lr_scheduler_parameters={"factor": 0.5, "patience": 10},
)

### Initialize the datamodule

The data module can take a `Path` or `str` to a folder or file, or a `np.ndarray`.

For custom types, you need to pass a read function and an extension_filter.

In [ ]:
train_data_module = CAREamicsTrainDataModule(
    train_data=train_path,
    val_data=val_path,
    data_type="tiff",  # to use np.ndarray, set data_type to "array"
    patch_size=(64, 64),
    axes="SYX",
    batch_size=128,
    dataloader_params={"num_workers": 4},
    use_n2v2=use_n2v2,
    struct_n2v_axis="none",  # choice between "horizontal", "vertical", or "none" (no structN2V)
    struct_n2v_span=7,
)

### Run training 

We need to specify the paths to training and validation data

In [ ]:
trainer = Trainer(max_epochs=1, default_root_dir="bsd_test")

In [ ]:
trainer.fit(model, datamodule=train_data_module)

### Define a prediction datamodule

In [ ]:
pred_data_module = CAREamicsPredictDataModule(
    pred_data=test_path,
    data_type="tiff",
    tile_size=(256, 256),
    tile_overlap=(48, 48),
    axes="YX",
    batch_size=1,
    tta_transforms=True,
)

### Run prediction

First, we want to use CAREamics prediction loop, which allows tiling:

In [ ]:
tiled_loop = CAREamicsPredictionLoop(trainer)
trainer.predict_loop = tiled_loop

Then, we predict using the datamodule.

In [ ]:
preds = trainer.predict(model, datamodule=pred_data_module)

### Visualize results and compute metrics


In [ ]:
# Create a list of ground truth images

gts = [tifffile.imread(f) for f in sorted(gt_path.glob("*.tiff"))]

In [ ]:
# Plot single image

image_idx = 0
_, subplot = plt.subplots(1, 2, figsize=(10, 10))

subplot[0].imshow(preds[image_idx].squeeze(), cmap="gray")
subplot[0].set_title("Prediction")
subplot[1].imshow(gts[image_idx], cmap="gray")
subplot[1].set_title("Ground truth")


# Calculate PSNR for single image
psnr_single = psnr(preds[image_idx].squeeze(), gts[image_idx].squeeze())
print(f"PSNR for image {image_idx}: {psnr_single}")

In [ ]:
# Plot single image

image_idx = 0
_, subplot = plt.subplots(1, 2, figsize=(10, 10))

subplot[0].imshow(preds[image_idx].squeeze(), cmap="gray")
subplot[0].set_title("Prediction")
subplot[1].imshow(gts[image_idx], cmap="gray")
subplot[1].set_title("Ground truth")

In [ ]:
# Calculate PSNR for single image

psnr_single = psnr(gts[image_idx], preds[image_idx].squeeze())
print(f"PSNR for image {image_idx}: {psnr_single}")

In [ ]:
psnr_total = 0

for pred, gt in zip(preds, gts):
    psnr_total += psnr(gt, pred.squeeze())

print(f"PSNR total: {psnr_total / len(preds)}")